In [104]:
!pip install gym
!pip install pygame
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [105]:
import gym
import numpy as np
import pygame
import time
import random
from gym import spaces
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# player piece colors [player1, player2]
colors = ['#FF0000', '#0000FF']

# paths for each tile 1-35
node_combinations = [ 
    [(0,3), (1,5), (2,6), (4,7)], [(0,4), (1,5), (2,6), (3,7)], [(0,3), (1,6), (2,5), (4,7)], [(0,6), (1,5), (2,4), (3,7)],
    [(0,1), (2,3), (4,5), (6,7)], [(0,4), (1,5), (2,3), (6,7)], [(0,6), (1,5), (2,3), (4,7)], [(0,5), (1,4), (2,7), (3,6)],
    [(0,5), (1,4), (2,6), (3,7)], [(0,3), (1,4), (2,5), (6,7)], [(0,6), (1,4), (2,5), (3,7)], [(0,5), (1,4), (2,3), (6,7)],
    [(0,2), (1,3), (4,6), (5,7)], [(0,2), (1,3), (4,5), (6,7)], [(0,5), (1,3), (2,7), (4,6)], [(0,6), (1,3), (2,7), (4,5)],
    [(0,4), (1,3), (2,6), (5,7)], [(0,5), (1,3), (2,6), (4,7)], [(0,4), (1,3), (2,5), (6,7)], [(0,6), (1,3), (2,5), (4,7)],
    [(0,5), (1,3), (2,4), (6,7)], [(0,6), (1,3), (2,4), (5,7)], [(0,3), (1,2), (4,7), (5,6)], [(0,3), (1,2), (4,6), (5,7)],
    [(0,3), (1,2), (4,5), (6,7)], [(0,4), (1,2), (3,7), (5,6)], [(0,5), (1,2), (3,7), (4,6)], [(0,6), (1,2), (3,7), (4,5)],
    [(0,4), (1,2), (3,6), (5,7)], [(0,5), (1,2), (3,6), (4,7)], [(0,4), (1,2), (3,5), (6,7)], [(0,6), (1,2), (3,5), (4,7)],
    [(0,5), (1,2), (3,4), (6,7)], [(0,6), (1,2), (3,4), (5,7)], [(0,7), (1,2), (3,4), (5,6)]
]


In [106]:
class Tile():
    def __init__(self, tile_num, tile_connections):
        self.tile_num = tile_num
        self.image = pygame.image.load("TsuroImages/" + str(tile_num) + ".png")
        self.image = pygame.transform.scale(self.image, (100, 100))
        self.tile_connections = tile_connections
    
    def move(self, current_node):
        next_node = 0
        next_player_tile = 0
        for connection in self.tile_connections:
            if current_node in connection:
                n1, n2 = connection
                if n1 == current_node:
                    next_node, next_player_tile, next_x, next_y = self.new_tile_node(n2)
                else:
                    next_node, next_player_tile, next_x, next_y = self.new_tile_node(n1)
                return next_node, next_player_tile, next_x, next_y
        raise Exception("Issue in moving players")
    
    # update number of times rotation should be applied to connections and image
    def rotate_tile(self, rotate):
        self.image = pygame.transform.rotate(self.image, rotate * -90)
        self.tile_connections = [tuple((element + (2 * rotate)) % 8 for element in couple ) for couple in self.tile_connections]
    
    def new_tile_node(self, current_node):
        next_node = 0
        next_x = 0
        next_y = 0
        next_player_tile = 0
        match current_node:
            case 0:
                next_node = 3
                next_player_tile = -1
                next_x = -1
            case 1:
                next_node = 6
                next_player_tile = -6
                next_y = -1
            case 2:
                next_node = 5
                next_player_tile = -6
                next_y = -1
            case 3:
                next_node = 0
                next_player_tile = 1
                next_x = 1
            case 4:
                next_node = 7
                next_player_tile = 1
                next_x = 1
            case 5:
                next_node = 2
                next_player_tile = 6
                next_y = 1
            case 6:
                next_node = 1
                next_player_tile = 6
                next_y = 1
            case 7:
                next_node = 4
                next_player_tile = -1
                next_x = -1
            case _:
                raise Exception("Issue in tile board")
                
        return next_node, next_player_tile, next_x, next_y
    

In [107]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size # 228
        self.action_size = action_size # 140
        self.memory = []
        self.gamma = 0.95   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, card, rotation, reward, next_state, done):
        action = (card, rotation)
        self.memory.append((np.array(state, dtype=object), action, reward, np.array(next_state, dtype=object), done))
        
    def act(self, state):
        hand_observation = state[0] # what cards the agent has - 1 x 3 array
        players_observation = state[1] # where each player is - 8 x 36 array
        board_observation = state[2] # which cards are placed where - 6x6 array
        current_player = state[3] # which player number the agent is (corresponds to number in players_observation) - int

        valid_moves = []
        for i, card in enumerate(hand_observation):
            for rotation in range(3):
                valid_moves.append((card, rotation))

        if np.random.rand() < self.epsilon:
            # Exploration
            move_idx = np.random.randint(len(valid_moves))
            card, rotation = valid_moves[move_idx]
        else:
            # Exploitation
            state = np.array(state)
            state = state.reshape((1, state.shape[0], state.shape[1], state.shape[2]))
            q_values = self.model.predict(state)
            valid_q_values = []
            for move in valid_moves:
                card_idx, rot_idx = move
                valid_q_values.append(q_values[0, card_idx * 4 + rot_idx])
            best_idx = np.argmax(valid_q_values)
            card, rotation = valid_moves[best_idx]

        return card, rotation

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            card, rotation = action
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)))
            target_f = self.model.predict(state)
            target_f[0][self.action_size * card + rotation] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay


    def load(self, file_name):
        self.model.load_weights(file_name)
        
    def save(self, file_name):
        self.model.save_weights(file_name)
        
    def get_epsilon(self):
        return self.epsilon
        
    def load_epsilon(self, file_name):
        with open(file_name, "r") as f:
            epsilon = float(f.readline())
        self.epsilon = epsilon

In [108]:
class TsuroEnv(gym.Env):
    #
    def __init__(self):
        self.current_player = 1
        self.num_tiles = 35
        self.tile_board_size = (6, 6) 
        self.player_board_size = (36,8)
        self.num_players = 2
        self.tiles = []
        for i in range(self.num_tiles):
            self.tiles.append(Tile(i, node_combinations[i]))
            
        self.remaining_tiles = []
        for i in range(self.num_tiles):
            self.remaining_tiles.append(i)
        random.shuffle(self.remaining_tiles)

        self.remaining_players = []
        for i in range(self.num_players):
            self.remaining_players.append(i+1)
        
        self.player_tiles = []
        for i in range(self.num_players):
            player_tiles = []
            for i in range(3):
                player_tiles.append(self.remaining_tiles.pop())
            self.player_tiles.append(player_tiles)
            
        self.tile_board = np.zeros(self.tile_board_size, dtype = int)
        self.player_board = np.zeros(self.player_board_size, dtype = int)

        self.action_space = spaces.Discrete(self.num_tiles*4)
        self.hand_observation_space = spaces.Box(low=0, high=35, shape=(3,))
        self.players_observation_space = spaces.Box(low=0, high=self.num_players, shape=(288,))
        self.board_observation_space = spaces.Box(low=0, high=35, shape=(36,))
        self.current_player_observation_space = spaces.Discrete(self.num_players)
        self.observation_space = spaces.Tuple((self.hand_observation_space, self.players_observation_space, self.board_observation_space, self.current_player_observation_space))
        
    # Resets the environment to default state
    def reset(self): 
        self.current_player = 1
        self.tile_board = np.zeros(self.tile_board_size, dtype = int)
        self.player_board = np.zeros(self.player_board_size, dtype = int)
        
        self.tiles = []
        for i in range(self.num_tiles):
            self.tiles.append(Tile(i, node_combinations[i]))
            
        self.remaining_tiles = []
        for i in range(self.num_tiles):
            self.remaining_tiles.append(i)
        random.shuffle(self.remaining_tiles)
            
        self.remaining_players = []
        for i in range(self.num_players):
            self.remaining_players.append(i+1)
            
        self.player_tiles = []
        for i in range(self.num_players):
            player_tiles = []
            for i in range(3):
                player_tiles.append(self.remaining_tiles.pop())
            self.player_tiles.append(player_tiles)
            
        #########################################
        #TODO: TESTING STUFF TO BE REMOVED LATER#
        #########################################
        for i in range(self.num_players):
            self.player_board[random.randint(0,5)][i+1] = i+1
            
        initial_obs = (np.array(self.player_tiles[self.current_player-1]), self.player_board.flatten(), self.tile_board.flatten(),  self.current_player)

        return initial_obs
    
    # Makes a move in the game based on inputs from player or AI
    def step(self, action, rotate):
        # Removes used tile and adds new tile from deck to hand
        self.player_tiles[self.current_player-1].remove(action)
        if len(self.remaining_tiles) > 0:
            self.player_tiles[self.current_player-1].append(self.remaining_tiles.pop())
        
        # Rotates tile (Only used by AI)
        self.tiles[action].rotate_tile(rotate)
        
        reward = 0
        self.place_tile(action+1)
        self.move_players()
        reward = self.reward_function()
        if self.game_is_over():
            done = 1
        else:
            done = 0
        self.current_player = self.next_player()
        observation = (np.array(self.player_tiles[self.current_player-1]), self.player_board.flatten(), self.tile_board.flatten(), self.current_player)
        return observation, reward, done, {}
    
    # Decides if the game is over
    def game_is_over(self):
        if len(self.remaining_players) <= 1:
            return True
        return False
    
    # Decides the reward (Only used for AI)
    def reward_function(self):
        if not self.game_is_over():
            return 1
        if self.game_is_over() and self.current_player in self.remaining_players:
            return 2
        return -1
    
    # Places tile in self.tile_board
    def place_tile(self, tile):
        tile_number, node_number = np.where(self.player_board == self.current_player)
        x, y = TsuroEnv.euclidean_division(self, tile_number)
        x = x[0]
        y = y[0]
        self.tile_board[x][y] += tile
    
    # Moves player piece in self.player_board
    def move_players(self):
        for player in self.remaining_players:
            tile_number, node_number = np.where(self.player_board == player)
            x, y = TsuroEnv.euclidean_division(self, tile_number)
            x = x[0]
            y = y[0]
            while self.tile_board[x][y] != 0:
                tile = self.tiles[(self.tile_board[x][y])-1]
                next_node, next_player_tile, next_x, next_y = tile.move(node_number)
                self.player_board[tile_number[0]][node_number[0]] = 0
                if ((tile_number[0] % 6 == 0) and ((tile_number[0] + next_player_tile) % 6 == 5)) or (tile_number[0] + next_player_tile < 0) or (tile_number[0] + next_player_tile > 35) or ((tile_number[0] % 6 == 5) and ((tile_number[0] + next_player_tile) % 6 == 0)):
                    self.remaining_players.remove(player)
                    break
                else:
                    self.player_board[tile_number[0] + next_player_tile][next_node] = player
                    x += next_x
                    y += next_y
                    tile_number, node_number = np.where(self.player_board == player)
    
    # Quotient and Remainder
    def euclidean_division(self, x, y = 6):
        return x % y, x // y
    
    # Decide whos turn it is
    def next_player(self):
        if len(self.remaining_players) == 0:
            return -1
        if self.current_player not in self.remaining_players:
            for player in self.remaining_players:
                if player > self.current_player:
                    return player
                else:
                    return self.remaining_players[0]
        return self.remaining_players[(self.remaining_players.index(self.current_player) + 1) % len(self.remaining_players)]
        
    # Render the environment
    def render(self):
        screen = pygame.display.set_mode((650, 750))
        screen.fill((255, 255, 255))

        # Draw the game board
        board = pygame.image.load("TsuroImages/board.png")
        board = pygame.transform.scale(board, (600, 600))
        screen.blit(board, (25,25))
        
        # Draw current players hand
        for i in range (len(self.player_tiles[self.current_player-1])):
            tile = self.player_tiles[self.current_player-1][i]
            screen.blit(self.tiles[tile].image, (75 + (i * 200), 635))
        
        # Draw the tiles on the board
        for x in range(self.tile_board_size[0]):
            for y in range(self.tile_board_size[1]):
                val = self.tile_board[x][y]
                if val != 0:
                    tile = self.tiles[val-1]
                    screen.blit(tile.image, (25 + x * 100, 25 + y * 100))
                    
        # Draw the players' pieces on the board
        for i in self.remaining_players:
            tile_number, node_number = np.where(self.player_board == i)
            y_add = 0
            x_add = 0
            y_mult = 0
            x_mult = 0
            
            match node_number[0]:
                case 0:
                    y_add = 35
                case 1:
                    x_add = 35
                case 2:
                    x_add = 70
                case 3:
                    x_add = 100
                    y_add = 35
                case 4:
                    x_add = 100
                    y_add = 70
                case 5:
                    x_add = 70
                    y_add = 100
                case 6:
                    x_add = 35
                    y_add = 100
                case 7:
                     y_add = 70
                case _:
                    raise Exception("Issue in drawing the player board")
                    
            if tile_number[0] != 0:
                x_mult, y_mult = TsuroEnv.euclidean_division(self, tile_number[0])
            
            pygame.draw.circle(screen, colors[i-1], (25 + x_add + (100 * x_mult), 25 + y_add + (100 * y_mult)), 5)
            
        # Draw text to show who won when game is over
        if self.game_is_over() or self.current_player == -1:
            font = pygame.font.Font('freesansbold.ttf', 32)
            text = font.render('Player ' + str(self.current_player) + ' wins', True, '#00FF00')
            textRect = text.get_rect()
            textRect.center = (650 // 2, 750 // 2)
            screen.blit(text, textRect)
            
        pygame.display.update()


In [109]:
def make_move(card, rotate):
        env.step(card, rotate)

def play():
    pygame.init()
    running = True
    env = TsuroEnv()
    state = env.reset()
    
    for i in range(env.num_players):
        temp = np.where(env.player_board == i+1)
        print("PLAYER " + str(i+1) + "'S STARTING POS: \nTile: " + str(temp[0]) + "    Node: " + str(temp[1]))
        
    while running:
        to_move = True
        env.render()
        print(env.player_tiles[0])
        print(env.player_tiles[1])
        while to_move:
            mouse = pygame.mouse.get_pos()
            if env.current_player == 1:
                if 75 + 100 > mouse[0] > 75 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][0]]
                                tile.rotate_tile(1)
                                env.render()
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            make_move(env.player_tiles[0][0], 0)
                            env.render()
                elif 275 + 100 > mouse[0] > 275 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][1]]
                                tile.rotate_tile(1)
                                env.render()
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            make_move(env.player_tiles[0][1], 0)
                            env.render()
                elif 475 + 100 > mouse[0] > 475 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][2]]
                                tile.rotate_tile(1)
                                env.render()
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            make_move(env.player_tiles[0][2], 0)
                            env.render()

            elif env.current_player == 2:
                time.sleep(2)
                count = random.randint(0,2)
                card = env.player_tiles[env.current_player-1][count]
                rotate = random.randint(0,3)
                env.step(card, rotate)
                env.render()
                to_move = False

            if env.current_player == -1 or env.game_is_over():
                env.render()
                print("Winner: Player " + str(env.current_player))
                running = False

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    pygame.quit()
                 
    if not running:
        env.render()
        
    while not running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()


In [110]:
def trainLoop():
    state_size = 328
    action_size = 140
    num_episodes = 1
    save_every = 1

    pygame.init()
    env = TsuroEnv()

    agent1 = DQNAgent(state_size, action_size)
    agent2 = DQNAgent(state_size, action_size)

    agent1Reward = 0
    agent2Reward = 0

    # restart training from blank weightings
    agent1.save("AgentWeights/agent1_weights.h5")
    agent2.save("AgentWeights/agent2_weights.h5")

    agent1.load("AgentWeights/agent1_weights.h5")
    agent2.load("AgentWeights/agent2_weights.h5")
    
    agent1.load_epsilon("AgentWeights/agent1_epsilon.txt")
    agent2.load_epsilon("AgentWeights/agent2_epsilon.txt")

    for episode in range(num_episodes):
        state = env.reset()
        env.render()
        done = False
        while not done:
            if env.current_player == 1:
                card, rotation = agent1.act(state)
                next_state, reward, done, _ = env.step(card, rotation)
                env.render()
                agent1Reward += reward
                #time.sleep(1)
                agent1.remember(state, card, rotation, reward, next_state, done)
                state = next_state
            elif env.current_player == 2:
                card, rotation = agent2.act(state)
                next_state, reward, done, _ = env.step(card, rotation)
                env.render()
                agent2Reward += reward
                #time.sleep(1)
                agent2.remember(state, card, rotation, reward, next_state, done)
                state = next_state
            else:
                print("ERROR")
                
        # save weights every 10 episodes
        if (episode + 1) % save_every == 0:
            agent1.save("AgentWeights/agent1_weights.h5")
            agent2.save("AgentWeights/agent2_weights.h5")
            
    batch_size1 = len(agent1.memory)
    batch_size2 = len(agent2.memory)
    
    agent1.replay(batch_size1)
    agent2.replay(batch_size2)
    
    with open("AgentWeights/rewards.txt", "a") as f:
        f.write(str(agent1Reward) + "," + str(agent2Reward) + "\n")
        
    with open("AgentWeights/agent1_epsilon.txt", "w") as f1:
        f1.write(str(agent1.get_epsilon()))
        
    with open("AgentWeights/agent2_epsilon.txt", "w") as f2:
        f2.write(str(agent2.get_epsilon()))

    agent1.save("AgentWeights/agent1_weights.h5")
    agent2.save("AgentWeights/agent2_weights.h5")

    print("Agent1's total reward for " + str(num_episodes) + " episodes: " + str(agent1Reward))
    print("Agent2's total reward for " + str(num_episodes) + " episodes: " + str(agent2Reward))

    pygame.quit()

In [111]:
def main():
    for i in range(1):
        trainLoop()
        print("loop count: " + str(i))
        
main()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).